In [1]:
import json
import datetime
import pandas as pd
import numpy as np
from collections import OrderedDict

###Load data

In [2]:
#Data from http://www.ultimaterankings.net/download.html
json_data=open('usau_database.json').read()
data=json.loads(json_data)

###Contents of data file

In [3]:
data.keys()

[u'tournaments', u'games', u'id', u'teams']

In [4]:
data['teams'][0]

{u'id': u'dHP4OF2933HXVGrIpWK%2f4FCcI6ithl6x98Nwnx3%2b6Ic',
 u'name': u'Alabama-Birmingham (Inferno)',
 u'usau_rating': {u'losses': 8, u'score': 1330.0, u'wins': 10}}

In [5]:
data['tournaments'][0]

{u'id': u'Heart-of-Texas-Huckfest-2016',
 u'name': u'Heart of Texas Huckfest 2016'}

In [6]:
data['games'][10]

{u'id': u'BJqUZviPseUsP%2bmZBMKRTUzuhlBsKCoHQ34pLgViuMg',
 u'local_datetime': u'2016-03-20 10:00',
 u'score': [13, 11],
 u'teams': [u'B6lUQxBw7%2fUGN0505cwf9CONoJwD%2fgEX7yiqLf%2b5yE0',
  u'nBwnp8VsMQk6mqcMw8QPOpDi%2bXJODhZUeLhRQ3VfSvY'],
 u'tournament': u'Trouble-in-Vegas-2016'}

In [7]:
data['id']

u'College-Men'

In [8]:
#Organize Data into Pandas Dataframes
games_df = pd.io.json.json_normalize(data['games'])
teams_df = pd.io.json.json_normalize(data['teams'])

In [9]:
#Get the unique timestamps and organize chronologically
timesort = pd.unique(games_df['local_datetime'])[np.argsort(pd.unique(games_df['local_datetime']))]

In [10]:
teams_dict = {}
ratings_dict = {}
for team in data['teams']:
    teams_dict[team['id']] = team['name'] #Link each team's hash id to their team name for easy reference
    ratings_dict[team['id']] = 1.0 #Every team's initial rating is 1.0

In [11]:
#Enter all game information into a dictionary callable by their date and time
games_dict = {}
for time_stamp in timesort:
    time_games = games_df[games_df['local_datetime'] == time_stamp]
    time_games = time_games[time_games['blacklisted_reason'] != 'blacklisted']
    date_list = []
    internal_dict = {}
    internal_dict['teams'] = np.asarray(time_games['teams'])
    internal_dict['scores'] = np.asarray(time_games['score'])
    games_dict[time_stamp] = internal_dict 

###Calculate Bradley-Terry Ratings

In [13]:
def expected_BT_win(rating_1, rating_2, score):
    team_1_expected = np.sum(score)*(rating_1/(rating_1+rating_2))
    team_2_expected = np.sum(score)*(rating_2/(rating_1+rating_2))
    return team_1_expected, team_2_expected

In [14]:
def first_order_BT(rating_1, rating_2, score):
    team_1_correction = np.sum(score)*(rating_2/np.square(rating_1+rating_2))
    team_2_correction = np.sum(score)*(rating_1/np.square(rating_1+rating_2))
    return team_1_correction, team_2_correction

In [15]:
wins_dict = {}
rating_corrections_dict = {}
approx_corr_max = 0.2

#Continue iterating until max correction is < 1e-7.
while approx_corr_max > 1e-7:
    approx_corr_max = 0.0
    zero_game_teams = []
    
    #Reset values
    for team_id in teams_dict.keys():
        wins_dict[team_id] = np.zeros(2) #[Actual, Expected]
        rating_corrections_dict[team_id] = 0.0 #Set the initial correction each time to 0.0
        
    #Run through all games collecting information on actual and expected wins
    prev_time_stamp = u'2016-01-01 12:00'
    length = len(timesort)+1
    for time_stamp in timesort:
        for score, teams in zip(games_dict[time_stamp]['scores'], games_dict[time_stamp]['teams']):
            old_rating_1 = ratings_dict[teams[0]]
            old_rating_2 = ratings_dict[teams[1]]
            exp_wins_1, exp_wins_2 = expected_BT_win(old_rating_1, old_rating_2, score)
            fo_corr_1, fo_corr_2 = first_order_BT(old_rating_1, old_rating_2, score)
            wins_dict[teams[0]] += [score[0], exp_wins_1]
            wins_dict[teams[1]] += [score[1], exp_wins_2]
            rating_corrections_dict[teams[0]] += fo_corr_1
            rating_corrections_dict[teams[1]] += fo_corr_2
        prev_time_stamp = time_stamp
        
    #Now use information collected to calculate rating corrections
    team_idx = 0
    for team_id in teams_dict.keys():
        if rating_corrections_dict[team_id] != 0.:
            approx_corr = (wins_dict[team_id][0] - wins_dict[team_id][1])*(1/(rating_corrections_dict[team_id]))
            if np.abs(approx_corr) > approx_corr_max:
                approx_corr_max = np.abs(approx_corr)
            if (ratings_dict[team_id] + approx_corr) < 0.001:
                ratings_dict[team_id] = 0.001
            else:
                ratings_dict[team_id] += approx_corr 
        else:
            zero_game_teams.append(team_idx) #Record which teams have zero games played
        team_idx+=1
    print approx_corr_max
print approx_corr_max, zero_game_teams

1.64912280702
1.62937224418
0.905751516695
1.50930415236
0.786691854515
2.14450979265
0.440643452563
1.23429512908
0.114543669106
0.100837973149
0.0884800670008
0.0774311322289
0.0675157397306
0.0587430190874
0.0511231514659
0.0446195516545
0.0389538517189
0.034017511317
0.029732934586
0.0260191847119
0.0228022712983
0.0200148384159
0.0175972580288
0.0154977635862
0.0136715694113
0.0120804188572
0.010691562463
0.00947712563677
0.0084133082779
0.00747981963986
0.00665930774084
0.00593692440349
0.00529992877286
0.00473737150234
0.00423981981268
0.00379913273789
0.00340826908458
0.00306112786878
0.00275241272815
0.00247751800699
0.00223243190439
0.0020136544351
0.00181812746536
0.00164317506967
0.00149332515178
0.00136393714575
0.00124638456231
0.00113950824844
0.00104227207137
0.000953748658266
0.000873106948544
0.000799601305743
0.000732561974683
0.000671386702055
0.000615533365302
0.000564513477517
0.000517886454798
0.000475254548696
0.000436258359999
0.000400572861227
0.00036790386556

In [902]:
final_ratings = []
for team in teams_dict.keys():
    final_ratings.append(ratings_dict[team])
final_rankings = np.argsort(final_ratings)[::-1]
i = 1
print "Final Top 50 Rankings"
for team_id in final_rankings[:50]:
    print str('#%i: ' %i), teams_dict.values()[team_id], ratings_dict.values()[team_id]
    i+=1

Final Top 50 Rankings
#1:  Massachusetts (Zoodisc) 3.93175098471
#2:  Minnesota (Grey Duck) 3.5112111374
#3:  Wisconsin (Hodags) 3.49247859968
#4:  Pittsburgh (En Sabah Nur) 3.22089833103
#5:  North Carolina (Darkside) 3.19484400972
#6:  Stanford (Stanford Bloodthirsty) 3.11534957695
#7:  Oregon (Ego) 3.04958772128
#8:  Florida (Florida) 3.04248629991
#9:  North Carolina-Wilmington (Seamen) 2.95001629926
#10:  British Columbia (Thunderbirds) 2.7012249931
#11:  Harvard (Red Line) 2.65775323639
#12:  Carleton College (Carleton College (CUT)) 2.60970252161
#13:  Washington (Sundodgers) 2.59454709927
#14:  Colorado (Mamabird) 2.56448225152
#15:  Georgia (Jojah) 2.55849469168
#16:  Brigham Young (CHI) 2.54971773854
#17:  Texas A&M (Dozen) 2.45935859327
#18:  Arkansas (Ludicrous Speed) 2.39230838674
#19:  Ohio State (Leadbelly) 2.3598919666
#20:  Cal Poly-SLO (SLOCORE) 2.31855160456
#21:  Texas (TUFF) 2.26602703659
#22:  Michigan (MagnUM) 2.24147786059
#23:  California-Santa Barbara (Black T

###Run simulations of Nationals

In [303]:
from scipy.stats import binom
def prob_win_game_15points(rating_1, rating_2):
    #Game to 15, cap at 17 win probability
    r = rating_1/(rating_1+rating_2)
    x = binom(28, r)
    overtime_16_14 = x.pmf(14)*(r**2) #Chance of 14-14 then 2 points in a row for team 1
    overtime_17_15 = x.pmf(14)*(2*(r*(1-r)))*(r**2) #Chance of 14-14 then 15-15 then 2 points in a row for team 1
    overtime_17_16 = x.pmf(14)*(2*(r*(1-r)))*(2*r*(1-r))*(r) #Chance of 15-15 then 16-16 then 1 point for team 1
    prob = np.sum(x.pmf(np.arange(15, 29))) + overtime_16_14 + overtime_17_15 + overtime_17_16
    #possible_scores = np.append(x.pmf, np.array(
    return prob

In [406]:
from scipy.misc import comb
from bisect import bisect
def calc_point_spread(rating_1, rating_2, draw_val):
    y = [0.0]
    r = rating_1/(rating_1+rating_2)
    score_diff_arr = np.arange(15,1,-1)
    
    overtime_16_14 = comb(28, 14)*(r**14)*((1-r)**14)*(r**2)
    overtime_17_15 = comb(28, 14)*(r**14)*((1-r)**14)*(2*(r*(1-r)))*(r**2) 
    overtime_17_16 = comb(28, 14)*(r**14)*((1-r)**14)*(2*(r*(1-r)))*(2*r*(1-r))*(r)
    
    for score_diff in score_diff_arr:
        x = comb(15+(15-score_diff)-1, 14, exact=True)*(r**15)*((1-r)**(15-score_diff))
        y.append(x+y[-1])
    y[-1]+=overtime_16_14
    y[-1]+=overtime_17_15
    y.append(overtime_17_16+y[-1])
    score_diff_arr = np.append(score_diff_arr, 1)
    if draw_val > y[-1]:
        draw_val = 1.0 - draw_val
    return score_diff_arr[bisect(y, draw_val)-1]

In [876]:
def three_team_tiebreak(poss_teams, team_wins):
    skip_score_diff = False
    pool_ranks = []
    score_diff = np.zeros(3)
    #In case one team beat the other two
    if poss_teams[1] in team_wins[str(poss_teams[0])][0]:
        if poss_teams[2] in team_wins[str(poss_teams[0])][0]:
            pool_ranks.append(poss_teams[0])
            if poss_teams[2] in team_wins[str(poss_teams[1])][0]:
                pool_ranks.append(poss_teams[1])
                pool_ranks.append(poss_teams[2])
                skip_score_diff = True
            else:
                pool_ranks.append(poss_teams[2])
                pool_ranks.append(poss_teams[1])
                skip_score_diff = True
        else:
            score_diff[0] += team_wins[str(poss_teams[0])][1][np.where(team_wins[str(poss_teams[0])][0] == poss_teams[1])[0]]
            score_diff[1] -= team_wins[str(poss_teams[0])][1][np.where(team_wins[str(poss_teams[0])][0] == poss_teams[1])[0]]
    elif poss_teams[2] in team_wins[str(poss_teams[0])][0]:
        score_diff[0] += team_wins[str(poss_teams[0])][1][np.where(team_wins[str(poss_teams[0])][0] == poss_teams[2])[0]]
        score_diff[2] -= team_wins[str(poss_teams[0])][1][np.where(team_wins[str(poss_teams[0])][0] == poss_teams[2])[0]]

    if poss_teams[0] in team_wins[str(poss_teams[1])][0]:
        if poss_teams[2] in team_wins[str(poss_teams[1])][0]:
            pool_ranks.append(poss_teams[1])
            if poss_teams[0] in team_wins[str(poss_teams[2])][0]:
                pool_ranks.append(poss_teams[2])
                pool_ranks.append(poss_teams[0])
                skip_score_diff = True
            else:
                pool_ranks.append(poss_teams[0])
                pool_ranks.append(poss_teams[2])
                skip_score_diff = True
        else:
            score_diff[1] += team_wins[str(poss_teams[1])][1][np.where(team_wins[str(poss_teams[1])][0] == poss_teams[0])[0]]
            score_diff[0] -= team_wins[str(poss_teams[1])][1][np.where(team_wins[str(poss_teams[1])][0] == poss_teams[0])[0]]
    elif poss_teams[2] in team_wins[str(poss_teams[1])][0]:
            score_diff[1] += team_wins[str(poss_teams[1])][1][np.where(team_wins[str(poss_teams[1])][0] == poss_teams[2])[0]]
            score_diff[2] -= team_wins[str(poss_teams[1])][1][np.where(team_wins[str(poss_teams[1])][0] == poss_teams[2])[0]]

    if poss_teams[1] in team_wins[str(poss_teams[2])][0]:
        if poss_teams[0] in team_wins[str(poss_teams[2])][0]:
            pool_ranks.append(poss_teams[2])
            if poss_teams[1] in team_wins[str(poss_teams[0])][0]:
                pool_ranks.append(poss_teams[0])
                pool_ranks.append(poss_teams[1])
                skip_score_diff = True
            else:
                pool_ranks.append(poss_teams[1])
                pool_ranks.append(poss_teams[0])
                skip_score_diff = True
        else:
            score_diff[2] += team_wins[str(poss_teams[2])][1][np.where(team_wins[str(poss_teams[2])][0] == poss_teams[1])[0]]
            score_diff[1] -= team_wins[str(poss_teams[2])][1][np.where(team_wins[str(poss_teams[2])][0] == poss_teams[1])[0]]
    elif poss_teams[0] in team_wins[str(poss_teams[2])][0]:
            score_diff[2] += team_wins[str(poss_teams[2])][1][np.where(team_wins[str(poss_teams[2])][0] == poss_teams[0])[0]]
            score_diff[0] -= team_wins[str(poss_teams[2])][1][np.where(team_wins[str(poss_teams[2])][0] == poss_teams[0])[0]]

    #Otherwise go to score differential
    if skip_score_diff == False:
        score_diff_rank = np.argsort(score_diff)[::-1]
        if len(np.unique(score_diff)) == 3:
            pool_ranks.append(poss_teams[score_diff_rank[0]])
            pool_ranks.append(poss_teams[score_diff_rank[1]])
            pool_ranks.append(poss_teams[score_diff_rank[2]])
        elif len(np.unique(score_diff)) == 2:
            if list(score_diff).count(np.max(score_diff)) == 1:
                pool_ranks.append(poss_teams[score_diff_rank[0]])
                if poss_teams[score_diff_rank[1]] in team_wins[str(poss_teams[score_diff_rank[2]])][0]:
                    pool_ranks.append(poss_teams[score_diff_rank[2]])
                    pool_ranks.append(poss_teams[score_diff_rank[1]])
                else:
                    pool_ranks.append(poss_teams[score_diff_rank[1]])
                    pool_ranks.append(poss_teams[score_diff_rank[2]])
            else:
                if poss_teams[score_diff_rank[1]] in team_wins[str(poss_teams[score_diff_rank[0]])][0]:
                    pool_ranks.append(poss_teams[score_diff_rank[0]])
                    pool_ranks.append(poss_teams[score_diff_rank[1]])
                else:
                    pool_ranks.append(poss_teams[score_diff_rank[1]])
                    pool_ranks.append(poss_teams[score_diff_rank[0]])
                pool_ranks.append(poss_teams[score_diff_rank[2]])
        elif len(np.unique(score_diff)) == 1:
            total_diff = [np.sum(team_wins[str(poss_teams[x])][1]) for x in xrange(3)]
            for team_num in xrange(5):
                if team_num not in poss_teams:
                    for val in xrange(3):
                        if poss_teams[val] in team_wins[str(team_num)][0]:
                            total_diff[val] -= team_wins[str(team_num)][1][np.where(team_wins[str(team_num)][0] == poss_teams[val])[0]]
            total_diff_rank = np.argsort(total_diff)[::-1]
            if len(np.unique(total_diff)) == 3:
                pool_ranks.append(poss_teams[total_diff_rank[0]])
                pool_ranks.append(poss_teams[total_diff_rank[1]])
                pool_ranks.append(poss_teams[total_diff_rank[2]])
            elif len(np.unique(total_diff)) == 2:
                if list(total_diff).count(np.max(total_diff)) == 1:
                    pool_ranks.append(poss_teams[total_diff_rank[0]])
                    if poss_teams[total_diff_rank[1]] in team_wins[str(poss_teams[total_diff_rank[2]])][0]:
                        pool_ranks.append(poss_teams[total_diff_rank[2]])
                        pool_ranks.append(poss_teams[total_diff_rank[1]])
                    else:
                        pool_ranks.append(poss_teams[total_diff_rank[1]])
                        pool_ranks.append(poss_teams[total_diff_rank[2]])
                else:
                    if poss_teams[total_diff_rank[1]] in team_wins[str(poss_teams[total_diff_rank[0]])][0]:
                        pool_ranks.append(poss_teams[total_diff_rank[0]])
                        pool_ranks.append(poss_teams[total_diff_rank[1]])
                    else:
                        pool_ranks.append(poss_teams[total_diff_rank[1]])
                        pool_ranks.append(poss_teams[total_diff_rank[0]])
                    pool_ranks.append(poss_teams[total_diff_rank[2]])    
            elif len(np.unique(total_diff)) == 1:
                print pool_ranks
                raise ValueError('New tiebreaker needed.')
    return pool_ranks

In [895]:
def pool_play_sim(r_dict, pool_teams):

    pool_wins = np.zeros(5)
    matchups = [[1,2], [3,4], [2,4], [3,5], [1,5], [1,3], [2,5], [1,4], [2,3], [4,5]] #order doesn't matter, just used pool A sched.
    team_wins = {str(x):[[],[]] for x in xrange(len(pool_teams))}
    for teams in matchups:
        win_1 = prob_win_game_15points(r_dict[pool_teams[teams[0]-1]], r_dict[pool_teams[teams[1]-1]])
        rand_draw = np.random.random()
        if rand_draw <= win_1:
            pool_wins[teams[0]-1]+=1
            point_spread = calc_point_spread(r_dict[pool_teams[teams[0]-1]], r_dict[pool_teams[teams[1]-1]], rand_draw)
            team_wins[str(teams[0]-1)][0].append(teams[1]-1)
            team_wins[str(teams[0]-1)][1].append(point_spread)
        else:
            pool_wins[teams[1]-1]+=1
            point_spread = calc_point_spread(r_dict[pool_teams[teams[1]-1]], r_dict[pool_teams[teams[0]-1]], rand_draw)
            team_wins[str(teams[1]-1)][0].append(teams[0]-1)
            team_wins[str(teams[1]-1)][1].append(point_spread)
    pool_ranks = []
#Tie breaker scenarios based upon https://www.cs.rochester.edu/u/ferguson/ultimate/upa-formats/upa-formats-basic.html#S2.5
    for wins in np.arange(4, -1, -1):
        poss_teams = np.where(pool_wins == wins)[0]
        if len(poss_teams) == 1:
            pool_ranks.append(poss_teams[0])
            
        if len(poss_teams) == 2:
            if poss_teams[1] in team_wins[str(poss_teams[0])][0]:
                pool_ranks.append(poss_teams[0])
                pool_ranks.append(poss_teams[1])
            else:
                pool_ranks.append(poss_teams[1])
                pool_ranks.append(poss_teams[0])
                
        if len(poss_teams) == 3:
            break_order = three_team_tiebreak(poss_teams, team_wins)
            for team_break in break_order:
                pool_ranks.append(team_break)
                        
        if len(poss_teams) == 5:
            total_diff = [np.sum(team_wins[str(poss_teams[x])][1]) for x in xrange(5)]
            for team_num in xrange(5):
                for val in xrange(5):
                    if poss_teams[val] in team_wins[str(team_num)][0]:
                        total_diff[val] -= team_wins[str(team_num)][1][np.where(team_wins[str(team_num)[0]][0] == poss_teams[val])[0]]
            total_diff_rank = np.argsort(total_diff)[::-1]
            
            if len(np.unique(total_diff)) == 5:
                    pool_ranks.append(poss_teams[total_diff_rank[0]])
                    pool_ranks.append(poss_teams[total_diff_rank[1]])
                    pool_ranks.append(poss_teams[total_diff_rank[2]])
                    pool_ranks.append(poss_teams[total_diff_rank[3]])
                    pool_ranks.append(poss_teams[total_diff_rank[4]])
                    
            if len(np.unique(total_diff)) == 4:
                if list(total_diff).count(np.max(total_diff)) == 1:
                    pool_ranks.append(poss_teams[total_diff_rank[0]])
                    if list(total_diff).count(np.sort(total_diff)[-2]) == 1:
                        pool_ranks.append(poss_teams[total_diff_rank[1]])
                        if list(total_diff).count(np.sort(total_diff)[-3]) == 1:
                            pool_ranks.append(poss_teams[total_diff_rank[2]])
                            if poss_teams[total_diff_rank[4]] in team_wins[str(poss_teams[total_diff_rank[3]])][0]:
                                pool_ranks.append(poss_teams[total_diff_rank[3]])
                                pool_ranks.append(poss_teams[total_diff_rank[4]])
                            else:
                                pool_ranks.append(poss_teams[total_diff_rank[4]])
                                pool_ranks.append(poss_teams[total_diff_rank[3]])
                        else:
                            if poss_teams[total_diff_rank[3]] in team_wins[str(poss_teams[total_diff_rank[2]])][0]:
                                pool_ranks.append(poss_teams[total_diff_rank[2]])
                                pool_ranks.append(poss_teams[total_diff_rank[3]])
                            else:
                                pool_ranks.append(poss_teams[total_diff_rank[3]])
                                pool_ranks.append(poss_teams[total_diff_rank[2]])
                            pool_ranks.append(poss_teams[total_diff_rank[4]])
                    else:
                        if poss_teams[total_diff_rank[2]] in team_wins[str(poss_teams[total_diff_rank[1]])][0]:
                            pool_ranks.append(poss_teams[total_diff_rank[1]])
                            pool_ranks.append(poss_teams[total_diff_rank[2]])
                        else:
                            pool_ranks.append(poss_teams[total_diff_rank[2]])
                            pool_ranks.append(poss_teams[total_diff_rank[1]])
                        pool_ranks.append(poss_teams[total_diff_rank[3]])
                        pool_ranks.append(poss_teams[total_diff_rank[4]])
                else:
                    if poss_teams[total_diff_rank[1]] in team_wins[str(poss_teams[total_diff_rank[0]])][0]:
                        pool_ranks.append(poss_teams[total_diff_rank[0]])
                        pool_ranks.append(poss_teams[total_diff_rank[1]])
                    else:
                        pool_ranks.append(poss_teams[total_diff_rank[1]])
                        pool_ranks.append(poss_teams[total_diff_rank[0]])
                    pool_ranks.append(poss_teams[total_diff_rank[2]])
                    pool_ranks.append(poss_teams[total_diff_rank[3]])
                    pool_ranks.append(poss_teams[total_diff_rank[4]])
            if len(np.unique(total_diff)) == 3:
                raise ValueError('test3')
                
            if len(np.unique(total_diff)) == 2:
                raise ValueError('test2')
                
            if len(np.unique(total_diff)) == 1:
                raise ValueError('test1')
                        
    if len(pool_ranks) != 5:
        print pool_ranks
        print team_wins
        raise ValueError('Not all ranked')
                
    return pool_wins, pool_ranks

In [832]:
def pick_winner(rating_1, rating_2):
    win_1 = prob_win_game_15points(rating_1, rating_2)
    rand_draw = np.random.random()
    if rand_draw <= win_1:
        winner = 0
    else:
        winner = 1
    return winner

In [824]:
def add_pool_rank(pool_ranks):
    pool_ranks = np.array(pool_ranks)
    pool_rank_matrix = np.zeros((5,5))
    for x in xrange(5):
        pool_rank_matrix[x,np.where(pool_ranks==x)[0]] += 1
    return pool_rank_matrix

In [171]:
#Nationals teams in pools
#Pool A
pool_a_teams = []
print "Pool A teams:"
for team_rank in [0, 14, 16, 12, 19]:
    pool_a_teams.append(teams_dict.keys()[final_rankings[team_rank]])
    print teams_dict[pool_a_teams[-1]]

pool_b_teams = []
print "\nPool B teams: "
for team_rank in [6, 4, 13, 24, 39]:
    pool_b_teams.append(teams_dict.keys()[final_rankings[team_rank]])
    print teams_dict[pool_b_teams[-1]]
    
pool_c_teams = []
print "\nPool C teams: "
for team_rank in [8, 2, 10, 21, 41]:
    pool_c_teams.append(teams_dict.keys()[final_rankings[team_rank]])
    print teams_dict[pool_c_teams[-1]]
    
pool_d_teams = []
print "\nPool D teams: "
for team_rank in [1, 3, 11, 23, 46]:
    pool_d_teams.append(teams_dict.keys()[final_rankings[team_rank]])
    print teams_dict[pool_d_teams[-1]]

Pool A teams:
Massachusetts (Zoodisc)
Georgia (Jojah)
Texas A&M (Dozen)
Washington (Sundodgers)
Cal Poly-SLO (SLOCORE)

Pool B teams: 
Oregon (Ego)
North Carolina (Darkside)
Colorado (Mamabird)
Florida State (DUF)
Case Western Reserve (Fighting Gobies[A])

Pool C teams: 
North Carolina-Wilmington (Seamen)
Wisconsin (Hodags)
Harvard (Red Line)
Michigan (MagnUM)
Connecticut (UConn Grind)

Pool D teams: 
Minnesota (Grey Duck)
Pittsburgh (En Sabah Nur)
Carleton College (Carleton College (CUT))
Auburn (Aetos)
Utah (Zion Curtain)


In [898]:
pool_a_totals = np.zeros(5)
pool_b_totals = np.zeros(5)
pool_c_totals = np.zeros(5)
pool_d_totals = np.zeros(5)
pool_a_end_rank = np.zeros((5,5))
pool_b_end_rank = np.zeros((5,5))
pool_c_end_rank = np.zeros((5,5))
pool_d_end_rank = np.zeros((5,5))
bracket_stats = {}
for pool in [pool_a_teams, pool_b_teams, pool_c_teams, pool_d_teams]:
    for team_id in pool:
        bracket_stats[team_id] = np.zeros(5)
total_sims = 10000
for i in xrange(total_sims):
    if i % 100 == 0:
        print "Simulation %i of %i" % (i, total_sims)
        
    #Simulate pool play    
    pool_a_wins, pool_a_ranks = pool_play_sim(ratings_dict, pool_a_teams)
    pool_a_totals += pool_a_wins
    pool_a_end_rank += add_pool_rank(pool_a_ranks)
    pool_b_wins, pool_b_ranks = pool_play_sim(ratings_dict, pool_b_teams)
    pool_b_totals += pool_b_wins
    pool_b_end_rank += add_pool_rank(pool_b_ranks)
    pool_c_wins, pool_c_ranks = pool_play_sim(ratings_dict, pool_c_teams)
    pool_c_totals += pool_c_wins
    pool_c_end_rank += add_pool_rank(pool_c_ranks)
    pool_d_wins, pool_d_ranks = pool_play_sim(ratings_dict, pool_d_teams)
    pool_d_totals += pool_d_wins
    pool_d_end_rank += add_pool_rank(pool_d_ranks)
    
    #Simulate Bracket
    #Pre-Quarters
    b2 = pool_b_teams[pool_b_ranks[1]]
    c3 = pool_c_teams[pool_c_ranks[2]]
    game_1_teams = [b2, c3]
    bracket_stats[b2][0] += 1
    bracket_stats[c3][0] += 1
    game_1_winner = game_1_teams[pick_winner(ratings_dict[game_1_teams[0]], ratings_dict[game_1_teams[1]])]
    
    c2 = pool_c_teams[pool_c_ranks[1]]
    b3 = pool_b_teams[pool_b_ranks[2]]
    game_2_teams = [c2, b3]
    bracket_stats[c2][0] += 1
    bracket_stats[b3][0] += 1
    game_2_winner = game_2_teams[pick_winner(ratings_dict[game_2_teams[0]], ratings_dict[game_2_teams[1]])]
    
    d2 = pool_d_teams[pool_d_ranks[1]]
    a3 = pool_a_teams[pool_a_ranks[2]]
    game_3_teams = [d2, a3]
    bracket_stats[d2][0] += 1
    bracket_stats[a3][0] += 1
    game_3_winner = game_3_teams[pick_winner(ratings_dict[game_3_teams[0]], ratings_dict[game_3_teams[1]])]
    
    a2 = pool_a_teams[pool_a_ranks[1]]
    d3 = pool_d_teams[pool_d_ranks[2]]
    game_4_teams = [a2, d3]
    bracket_stats[a2][0] += 1
    bracket_stats[d3][0] += 1
    game_4_winner = game_4_teams[pick_winner(ratings_dict[game_4_teams[0]], ratings_dict[game_4_teams[1]])]
    
    #Quarters
    q1_1 = game_1_winner
    q1_2 = pool_a_teams[pool_a_ranks[0]]
    q1_teams = [q1_1, q1_2]
    bracket_stats[q1_1][1] += 1
    bracket_stats[q1_2][0:2] += 1
    q1_winner = q1_teams[pick_winner(ratings_dict[q1_teams[0]], ratings_dict[q1_teams[1]])]

    q2_1 = game_2_winner
    q2_2 = pool_d_teams[pool_d_ranks[0]]
    q2_teams = [q2_1, q2_2]
    bracket_stats[q2_1][1] += 1
    bracket_stats[q2_2][0:2] += 1
    q2_winner = q2_teams[pick_winner(ratings_dict[q2_teams[0]], ratings_dict[q2_teams[1]])]

    q3_1 = game_3_winner
    q3_2 = pool_c_teams[pool_c_ranks[0]]
    q3_teams = [q3_1, q3_2]
    bracket_stats[q3_1][1] += 1
    bracket_stats[q3_2][0:2] += 1
    q3_winner = q3_teams[pick_winner(ratings_dict[q3_teams[0]], ratings_dict[q3_teams[1]])]

    q4_1 = game_4_winner
    q4_2 = pool_b_teams[pool_b_ranks[0]]
    q4_teams = [q4_1, q4_2]
    bracket_stats[q4_1][1] += 1
    bracket_stats[q4_2][0:2] += 1
    q4_winner = q4_teams[pick_winner(ratings_dict[q4_teams[0]], ratings_dict[q4_teams[1]])]
    
    #Semis
    s1_1 = q1_winner
    s1_2 = q2_winner
    s1_teams = [s1_1, s1_2]
    bracket_stats[s1_1][2]+=1
    bracket_stats[s1_2][2]+=1
    s1_winner = s1_teams[pick_winner(ratings_dict[s1_teams[0]], ratings_dict[s1_teams[1]])]
    
    s2_1 = q3_winner
    s2_2 = q4_winner
    s2_teams = [s2_1, s2_2]
    bracket_stats[s2_1][2]+=1
    bracket_stats[s2_2][2]+=1
    s2_winner = s2_teams[pick_winner(ratings_dict[s2_teams[0]], ratings_dict[s2_teams[1]])]
    
    #Final
    f_1 = s1_winner
    f_2 = s2_winner
    f_teams = [f_1, f_2]
    bracket_stats[f_1][3]+=1
    bracket_stats[f_2][3]+=1
    champ = f_teams[pick_winner(ratings_dict[f_teams[0]], ratings_dict[f_teams[1]])]
    bracket_stats[champ][4]+=1

Simulation 0 of 10000
Simulation 100 of 10000
Simulation 200 of 10000
Simulation 300 of 10000
Simulation 400 of 10000
Simulation 500 of 10000
Simulation 600 of 10000
Simulation 700 of 10000
Simulation 800 of 10000
Simulation 900 of 10000
Simulation 1000 of 10000
Simulation 1100 of 10000
Simulation 1200 of 10000
Simulation 1300 of 10000
Simulation 1400 of 10000
Simulation 1500 of 10000
Simulation 1600 of 10000
Simulation 1700 of 10000
Simulation 1800 of 10000
Simulation 1900 of 10000
Simulation 2000 of 10000
Simulation 2100 of 10000
Simulation 2200 of 10000
Simulation 2300 of 10000
Simulation 2400 of 10000
Simulation 2500 of 10000
Simulation 2600 of 10000
Simulation 2700 of 10000
Simulation 2800 of 10000
Simulation 2900 of 10000
Simulation 3000 of 10000
Simulation 3100 of 10000
Simulation 3200 of 10000
Simulation 3300 of 10000
Simulation 3400 of 10000
Simulation 3500 of 10000
Simulation 3600 of 10000
Simulation 3700 of 10000
Simulation 3800 of 10000
Simulation 3900 of 10000
Simulation 4

In [899]:
print pool_a_totals/10000
print pool_b_totals/10000
print pool_c_totals/10000
print pool_d_totals/10000

[ 3.5834  1.7457  1.5603  1.8346  1.276 ]
[ 2.8434  3.0738  2.0312  1.2926  0.759 ]
[ 2.6049  3.2964  2.1191  1.3284  0.6512]
[ 3.2648  2.9276  2.0241  1.3069  0.4766]


In [900]:
print pool_a_end_rank/10000
print pool_b_end_rank/10000
print pool_c_end_rank/10000
print pool_d_end_rank/10000

[[ 0.812   0.1447  0.0308  0.0095  0.003 ]
 [ 0.0565  0.2497  0.2661  0.225   0.2027]
 [ 0.0415  0.2031  0.24    0.2577  0.2577]
 [ 0.067   0.2692  0.2665  0.2272  0.1701]
 [ 0.023   0.1333  0.1966  0.2806  0.3665]]
[[ 0.3675  0.3603  0.1881  0.0627  0.0214]
 [ 0.484   0.3204  0.1405  0.0453  0.0098]
 [ 0.1133  0.2107  0.3661  0.2137  0.0962]
 [ 0.0284  0.0795  0.2079  0.3888  0.2954]
 [ 0.0068  0.0291  0.0974  0.2895  0.5772]]
[[ 0.242   0.3778  0.2533  0.0965  0.0304]
 [ 0.6124  0.2647  0.0923  0.0268  0.0038]
 [ 0.119   0.2513  0.3598  0.195   0.0749]
 [ 0.0239  0.0838  0.2205  0.412   0.2598]
 [ 0.0027  0.0224  0.0741  0.2697  0.6311]]
[[ 0.5469  0.327   0.0997  0.0236  0.0028]
 [ 0.3472  0.4113  0.1817  0.0526  0.0072]
 [ 0.0841  0.1943  0.4415  0.2184  0.0617]
 [ 0.0208  0.0599  0.2289  0.4695  0.2209]
 [ 0.001   0.0075  0.0482  0.2359  0.7074]]


In [901]:
for bracket_id in bracket_stats.keys():
    print teams_dict[bracket_id], bracket_stats[bracket_id]/10000.

Wisconsin (Hodags) [ 0.9694  0.8787  0.5336  0.3243  0.167 ]
Massachusetts (Zoodisc) [ 0.9875  0.9607  0.7582  0.5254  0.371 ]
Texas A&M (Dozen) [ 0.4846  0.197   0.0552  0.014   0.0024]
Case Western Reserve (Fighting Gobies[A]) [ 0.1333  0.0249  0.0023  0.0002  0.    ]
North Carolina-Wilmington (Seamen) [ 0.8731  0.6161  0.2484  0.0996  0.0324]
Michigan (MagnUM) [  3.28200000e-01   1.16900000e-01   1.50000000e-02   2.10000000e-03
   3.00000000e-04]
Cal Poly-SLO (SLOCORE) [  3.52900000e-01   1.16800000e-01   2.51000000e-02   5.90000000e-03
   1.00000000e-04]
Harvard (Red Line) [ 0.7301  0.4079  0.1183  0.0334  0.0078]
North Carolina (Darkside) [ 0.9449  0.7873  0.4296  0.202   0.0841]
Florida State (DUF) [  3.15800000e-01   1.03100000e-01   1.67000000e-02   2.50000000e-03
   1.00000000e-04]
Carleton College (Carleton College (CUT)) [ 0.7199  0.399   0.1304  0.0345  0.0064]
Oregon (Ego) [ 0.9159  0.6981  0.3377  0.1435  0.05  ]
Colorado (Mamabird) [ 0.6901  0.3488  0.099   0.0243  0.005

In [908]:
i=1
f = open('ratings_2016_reg_season.dat', 'w')
for team_id in final_rankings:
    f.write(str(teams_dict.values()[team_id].encode('UTF-8') + ',' + str(ratings_dict.values()[team_id]) + '\n'))
    i+=1
f.close()

In [922]:
f = open('bracket_stats_2016.dat', 'w')
pool_names = ['Pool_A', 'Pool_B', 'Pool_C', 'Pool_D']
pool_lists = [pool_a_teams, pool_b_teams, pool_c_teams, pool_d_teams]
for pool_name, pool_list in zip(pool_names, pool_lists):
    f.write(str('#'+pool_name+'\n'))
    for pool_tm in pool_list:
        f.write(str(teams_dict[pool_tm].encode('UTF-8') + ',%.4f,%.4f,%.4f,%.4f,%.4f\n') % (bracket_stats[pool_tm][0]/10000,
                                                                                        bracket_stats[pool_tm][1]/10000,
                                                                                        bracket_stats[pool_tm][2]/10000,
                                                                                        bracket_stats[pool_tm][3]/10000,
                                                                                        bracket_stats[pool_tm][4]/10000))
f.close()

In [942]:
f = open('pool_placement_2016.dat', 'w')
pool_names = ['Pool_A', 'Pool_B', 'Pool_C', 'Pool_D']
pool_lists = [pool_a_teams, pool_b_teams, pool_c_teams, pool_d_teams]
pool_rank_lists = [pool_a_end_rank, pool_b_end_rank, pool_c_end_rank, pool_d_end_rank]
for pool_name, pool_list, pool_rank_list in zip(pool_names, pool_lists, pool_rank_lists):
    f.write(str('#'+pool_name+'\n'))
    i = 0
    for pool_tm in pool_list:
        f.write(str(teams_dict[pool_tm].encode('UTF-8') + ',%.4f,%.4f,%.4f,%.4f,%.4f\n') % (pool_rank_list[i][0]/10000,
                                                                                        pool_rank_list[i][1]/10000,
                                                                                        pool_rank_list[i][2]/10000,
                                                                                        pool_rank_list[i][3]/10000,
                                                                                        pool_rank_list[i][4]/10000))
        i+=1
f.close()

In [947]:
f = open('pool_wins_2016.dat', 'w')
pool_names = ['Pool_A', 'Pool_B', 'Pool_C', 'Pool_D']
pool_lists = [pool_a_teams, pool_b_teams, pool_c_teams, pool_d_teams]
pool_rank_lists = [pool_a_totals, pool_b_totals, pool_c_totals, pool_d_totals]
for pool_name, pool_list, pool_rank_list in zip(pool_names, pool_lists, pool_rank_lists):
    f.write(str('#'+pool_name+'\n'))
    i = 0
    for pool_tm in pool_list:
        f.write(str(teams_dict[pool_tm].encode('UTF-8') + ',%.4f\n') % (pool_rank_list[i]/10000))
        i+=1
f.close()

In [948]:
np.genfromtxt('pool_wins_2016.dat', delimiter=',', dtype=None)

array([('Massachusetts (Zoodisc)', 3.5834), ('Georgia (Jojah)', 1.7457),
       ('Texas A&M (Dozen)', 1.5603), ('Washington (Sundodgers)', 1.8346),
       ('Cal Poly-SLO (SLOCORE)', 1.276), ('Oregon (Ego)', 2.8434),
       ('North Carolina (Darkside)', 3.0738),
       ('Colorado (Mamabird)', 2.0312), ('Florida State (DUF)', 1.2926),
       ('Case Western Reserve (Fighting Gobies[A])', 0.759),
       ('North Carolina-Wilmington (Seamen)', 2.6049),
       ('Wisconsin (Hodags)', 3.2964), ('Harvard (Red Line)', 2.1191),
       ('Michigan (MagnUM)', 1.3284),
       ('Connecticut (UConn Grind)', 0.6512),
       ('Minnesota (Grey Duck)', 3.2648),
       ('Pittsburgh (En Sabah Nur)', 2.9276),
       ('Carleton College (Carleton College (CUT))', 2.0241),
       ('Auburn (Aetos)', 1.3069), ('Utah (Zion Curtain)', 0.4766)], 
      dtype=[('f0', 'S41'), ('f1', '<f8')])